In [36]:
# 單純預測隔天漲跌(漲,跌,不動)目前結果大概都落在50~60% 準確度
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn import ensemble
import numpy as np
import pandas as pd


In [37]:
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
import seaborn as sns

myfont = FontProperties(fname=r'msj.ttf',size=14)
sns.set(font=myfont.get_family())
sns.set_style("whitegrid", {"font.sans-serif": ['Microsoft JhengHei']})
%matplotlib inline 

In [38]:
# input data
feature_df = pd.read_csv("data/company/2317.csv", encoding='big5', thousands=',', index_col="日期")
feature_df.head()

,證券代號,證券名稱,產業別,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,...,投信賣出股數,投信買賣超股數,自營商買賣超股數,自營商買進股數(自行買賣),自營商賣出股數(自行買賣),自營商買賣超股數(自行買賣),自營商買進股數(避險),自營商賣出股數(避險),自營商買賣超股數(避險),三大法人買賣超股數
日期,,,,,,,,,,,,,,,,,,,,,
20190102,2317,鴻海,其他電子業,16775306,7968,1182131473,71.4,71.4,69.9,70.1,...,0,26000,-1394000,38000,1132000,-1094000,117000,417000,-300000,-6104550
20190103,2317,鴻海,其他電子業,36659461,17345,2526323765,69.0,69.4,68.5,68.9,...,600000,-600000,181000,12000,28000,-16000,532000,335000,197000,-10416196
20190104,2317,鴻海,其他電子業,37313571,18110,2520135828,68.2,68.2,67.0,67.6,...,1606805,-1573805,438000,51000,0,51000,839000,452000,387000,-7633271
20190107,2317,鴻海,其他電子業,24084557,11167,1661796863,68.7,69.4,68.3,69.3,...,355000,-355000,1344000,311000,140000,171000,1369000,196000,1173000,4579348
20190108,2317,鴻海,其他電子業,18303188,8027,1258685500,69.6,69.6,68.2,68.6,...,0,0,-224000,0,122000,-122000,208000,310000,-102000,904293


In [39]:
# data clean
feature_df = feature_df.drop(["證券代號", "證券名稱", "產業別"], axis=1)
feature_df["漲跌(+/-)"] = feature_df["漲跌(+/-)"].map({'+':1, '-':-1, 'X':0}).fillna(method="ffill")
feature_df = feature_df.applymap(lambda x: None if x == "--" else x).fillna(method="ffill")

In [40]:
# 看起來有關的：最後揭示買量、三大法人 這兩個最有關係
corr_df = feature_df.shift(1).drop(20190102).astype(float).corr()["漲跌(+/-)"]
# corr_df.where(corr_df <= -0.2)
corr_df

成交股數                 0.026739
成交筆數                 0.016470
成交金額                 0.027872
開盤價                 -0.032755
最高價                  0.010852
最低價                  0.020033
收盤價                  0.066833
漲跌(+/-)              1.000000
漲跌價差                 0.004164
最後揭示買價               0.064570
最後揭示買量              -0.243170
最後揭示賣價               0.056537
最後揭示賣量               0.019259
本益比                  0.041113
外陸資買進股數(不含外資自營商)     0.097088
外陸資賣出股數(不含外資自營商)    -0.209008
外陸資買賣超股數(不含外資自營商)    0.434596
外資自營商買進股數                 NaN
外資自營商賣出股數                 NaN
外資自營商買賣超股數                NaN
投信買進股數               0.169797
投信賣出股數              -0.067390
投信買賣超股數              0.170796
自營商買賣超股數             0.313897
自營商買進股數(自行買賣)        0.187802
自營商賣出股數(自行買賣)       -0.006923
自營商買賣超股數(自行買賣)       0.201754
自營商買進股數(避險)          0.336042
自營商賣出股數(避險)         -0.148003
自營商買賣超股數(避險)         0.342400
三大法人買賣超股數            0.469648
Name: 漲跌(+/-), dtype: float64

In [41]:
# split x,y
Y = feature_df["漲跌(+/-)"][1:]
# X = feature_df.drop(["漲跌(+/-)"], axis=1)[:-1]
X = feature_df[["開盤價", "收盤價" ,"三大法人買賣超股數"]]
# 用前一日的資料當作這次的feature
X = X.shift(1)
# 一階差分
X["開盤'"] = X["開盤價"] - X.shift(1)["開盤價"]
X["收盤'"] = X["收盤價"] - X.shift(1)["收盤價"]
X["三大'"] = X["三大法人買賣超股數"] - X.shift(1)["三大法人買賣超股數"]
# 5日平均
X["開盤a"] = X["開盤價"]
X["收盤a"] = X["收盤價"]
X["三大a"] = X["三大法人買賣超股數"]
for i in range(1, 5):
    X["開盤a"] += X.shift(i)["開盤價"]
    X["收盤a"] += X.shift(i)["收盤價"]
    X["三大a"] += X.shift(i)["三大法人買賣超股數"]
X["開盤a"] /= 5
X["收盤a"] /= 5
X["三大a"] /= 5
# 20日平均
X["開盤a20"] = X["開盤價"]
X["收盤a20"] = X["收盤價"]
X["三大a20"] = X["三大法人買賣超股數"]
for i in range(1, 20):
    X["開盤a20"] += X.shift(i)["開盤價"]
    X["收盤a20"] += X.shift(i)["收盤價"]
    X["三大a20"] += X.shift(i)["三大法人買賣超股數"]
X["開盤a20"] /= 20
X["收盤a20"] /= 20
X["三大a20"] /= 20

In [42]:
X.head(10)

,開盤價,收盤價,三大法人買賣超股數,開盤',收盤',三大',開盤a,收盤a,三大a,開盤a20,收盤a20,三大a20
日期,,,,,,,,,,,,
20190102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20190103,71.4,70.1,-6104550.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20190104,69.0,68.9,-10416196.0,-2.4,-1.2,-4311646.0,NaN,NaN,NaN,NaN,NaN,NaN
20190107,68.2,67.6,-7633271.0,-0.8,-1.3,2782925.0,NaN,NaN,NaN,NaN,NaN,NaN
20190108,68.7,69.3,4579348.0,0.5,1.7,12212619.0,NaN,NaN,NaN,NaN,NaN,NaN
20190109,69.6,68.6,904293.0,0.9,-0.7,-3675055.0,69.38,68.90,-3734075.2,NaN,NaN,NaN
20190110,68.9,69.9,12917708.0,-0.7,1.3,12013415.0,68.88,68.86,70376.4,NaN,NaN,NaN
20190111,69.8,69.3,-3641720.0,0.9,-0.6,-16559428.0,69.04,68.94,1425271.6,NaN,NaN,NaN
20190114,69.8,69.4,-1793782.0,0.0,0.1,1847938.0,69.36,69.30,2593169.4,NaN,NaN,NaN


In [43]:
X["漲跌"] = feature_df["漲跌(+/-)"]
X = X.dropna(axis=0, how="any")

In [44]:
X.astype(float).corr()["漲跌"]

開盤價         -0.120027
收盤價         -0.096976
三大法人買賣超股數    0.108229
開盤'          0.034128
收盤'          0.203787
三大'          0.216825
開盤a         -0.147715
收盤a         -0.144975
三大a          0.021669
開盤a20       -0.240721
收盤a20       -0.239847
三大a20       -0.119073
漲跌           1.000000
Name: 漲跌, dtype: float64

In [45]:
Y = X["漲跌"]
X = X.drop("漲跌", axis = 1)

In [46]:
# feature selection
# sel = VarianceThreshold(threshold=0.8)
# x = sel.fit_transform(X)

# split train test
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.33)

In [47]:
#k-NN
knn =  KNeighborsClassifier(n_neighbors=3)
knn.fit(x_train, y_train) 

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=3, p=2,
           weights='uniform')

In [48]:
knn.score(x_test, y_test)

0.5348837209302325

In [49]:
# decsiion tree
dt = tree.DecisionTreeClassifier()
dt = dt.fit(x_train, y_train)

In [50]:
dt.score(x_test, y_test)

0.5581395348837209

In [51]:
# random forest
rf = ensemble.RandomForestClassifier(n_estimators=50)
rf = rf.fit(x_train, y_train)


In [52]:
rf.score(x_test, y_test)

0.627906976744186